In [52]:
%pylab inline
import networkx as nx
import torch

import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

Populating the interactive namespace from numpy and matplotlib


In [53]:
edges=np.loadtxt("edges.txt",dtype=np.int32)
nodes=np.loadtxt("nodes.txt",dtype=np.int32)
G=nx.Graph()

for i in range(4):
    G.add_nodes_from(nodes[nodes[:,1]==i][:,0],labels=i)
G.add_edges_from(edges)

In [54]:
nodes=G.nodes(data=True)
values=[]
for i in range(1,G.number_of_nodes()+1):
    values.append(nodes[i]['labels'])
edge_index=torch.from_numpy(np.array(G.edges()))
y=torch.from_numpy(np.array(values))
x=torch.eye(G.number_of_nodes())

In [55]:
data = Data(x=x, edge_index=edge_index,y=y)

In [56]:
data

Data(edge_index=[77, 2], x=[34, 34], y=[34])

In [57]:
data.y==0

tensor([0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.uint8)

In [60]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.module=torch.nn.Sequential(
            GCNConv(data.num_features, 16),
            torch.nn.ReLU(),
            GCNConv(16, data.num_classes),
            torch.nn.LogSoftmax(dim=1)
        )
    def forward(self, data):
        return self.module((data.x, data.edge_index)


In [41]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [42]:
data.to(device)

RuntimeError: cuda runtime error (59) : device-side assert triggered at /pytorch/aten/src/THC/generic/THCTensorCopy.cpp:20

In [22]:
model, data = Net().to(device), data.to(device)

RuntimeError: cuda runtime error (59) : device-side assert triggered at /pytorch/aten/src/THC/generic/THCTensorCopy.cpp:20

In [15]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [43]:
y

tensor([1, 1, 2, 1, 0, 0, 0, 1, 3, 2, 0, 1, 1, 1, 3, 3, 0, 1, 3, 1, 3, 1, 3, 3,
        2, 2, 3, 2, 2, 3, 3, 2, 3, 3])

In [16]:
output=model(data)

RuntimeError: Expected object of type torch.FloatTensor but found type torch.cuda.FloatTensor for argument #2 'mat2'

In [ ]:
for epoch in range(1, 101):
    output=model(data)